In [1]:
# ZZ
import csv
import numpy as np
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import pickle
import pandas as pd
#import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
import math

In [2]:
# data_dir='data/'
data_dir = r"C:\work\dev\dECMT_src\data_all\COVID19_Data\Current\\"

#table='REACT_COVID_Demographics'
#table='REACT_COVID_Demographics_20200506'
#table='REACT_Events'
#table='REACT_LabResults'
#table='REACT_PharmacyData'
#table='REACT_UHSCOVIDTest_processed'
#table='REACT_Vitalsigns_Categorical'
#table='REACT_Vitalsigns_Numeric'

In [3]:
#df=pd.read_csv(data_dir+table+'.csv')
#df.head(20)

In [4]:
# df.info()

Extract AIR_ventilator information from table REACT_Events

In [5]:
air_ventilator_degree=[
    'Air - Not Supported',
    'Nasal Specs',
    'Face Mask',
    'Venturi Mask',
    'Trachy Mask',
    'Non-Rebreath Mask',
    'Optiflow / Hi Flow',
    'NIV - CPAP nasal mask',
    'NIV - CPAP face mask',
    'NIV - CPAP full face mask',
    'NIV - BIPAP nasal mask',
    'NIV - BIPAP face mask',
    'NIV - BIPAP full face mask',
    'Invasive Ventilation'
]
air_ventilator_degree_dic=dict([(air_ventilator_degree[i],float(i)) for i in range(len(air_ventilator_degree))])
air_ventilator_degree_dic

{'Air - Not Supported': 0.0,
 'Nasal Specs': 1.0,
 'Face Mask': 2.0,
 'Venturi Mask': 3.0,
 'Trachy Mask': 4.0,
 'Non-Rebreath Mask': 5.0,
 'Optiflow / Hi Flow': 6.0,
 'NIV - CPAP nasal mask': 7.0,
 'NIV - CPAP face mask': 8.0,
 'NIV - CPAP full face mask': 9.0,
 'NIV - BIPAP nasal mask': 10.0,
 'NIV - BIPAP face mask': 11.0,
 'NIV - BIPAP full face mask': 12.0,
 'Invasive Ventilation': 13.0}

In [6]:
df_REACT_Vitalsigns_Categorical=pd.read_csv(data_dir+r'\REACT_Vitalsigns_Categorical'+'.csv')
#df_REACT_Vitalsigns_Categorical.info()

In [7]:
studyid_air_ventilator_dic=dict()
for studyid,air_vent in df_REACT_Vitalsigns_Categorical[df_REACT_Vitalsigns_Categorical.VALUE.isin(air_ventilator_degree_dic)][['STUDY_ID','VALUE']].values:
    if studyid not in studyid_air_ventilator_dic or studyid_air_ventilator_dic[studyid]<air_ventilator_degree_dic[air_vent]:
        studyid_air_ventilator_dic[studyid]=air_ventilator_degree_dic[air_vent]
df_studyid_air_vent=pd.DataFrame(
    data=[[key,studyid_air_ventilator_dic[key]] for key in studyid_air_ventilator_dic],
    columns=['STUDY_ID','AIR_VENT_DEGREE']
)
#df_studyid_air_vent

Extract ITU_DAYS information from table REACT_Events

In [8]:
data_dir+'REACT_Events'+'.csv'

'C:\\work\\dev\\dECMT_src\\data_all\\COVID19_Data\\Current\\\\REACT_Events.csv'

In [9]:
df_REACT_Events=pd.read_csv(data_dir+r'\REACT_Events'+'.csv')
#df_REACT_Events.info()

In [10]:
from datetime import date

def str2date(ss):
    day,month,year=ss.split('/')
    return date(int(year),int(month),int(day))

max_date=max([str2date(x) for x in df_REACT_Events[df_REACT_Events.END_DATE.notnull()].END_DATE.unique()])
max_date=str(max_date.day)+'/'+str(max_date.month)+'/'+str(max_date.year)

#max_date

In [11]:
studyid_ITU_lst=[]
for studyid,start_date,end_date in df_REACT_Events[df_REACT_Events.EVENT_TYPE=='ITU'][['STUDY_ID','START_DATE','END_DATE']].fillna({'END_DATE':max_date}).values:
    start_day,start_month,start_year=map(int,start_date.split('/'))
    end_day,end_month,end_year=map(int,end_date.split('/'))
    days=int((date(end_year,end_month,end_day)-date(start_year,start_month,start_day)).days)
    studyid_ITU_lst.append([studyid,days])
df_studyid_ITU=pd.DataFrame(data=studyid_ITU_lst,columns=['STUDY_ID','ITU_DAYS'])
#df_studyid_ITU

load table REACT\_COVID\_Demographics

In [12]:
df_demographics=pd.read_csv(data_dir+'REACT_COVID_Demographics'+'.csv')
#df_demographics.head(10)

In [13]:
#df_demographics.info()

In [14]:
#len(df_demographics.STUDY_ID.unique())

In [15]:
#df_demographics[['STUDY_ID','PATIENT_AGE','GENDER','ETHNIC_GROUP','IS_PREGNANT']]

load table REACT_LabResults

In [16]:
df_labresults=pd.read_csv(data_dir+'REACT_LabResults'+'.csv')
#df_labresults.head(10)

In [17]:
#df_labresults.info()

In [18]:
#len(df_labresults.STUDY_ID.unique())

In [19]:
testcode_lst=df_labresults.REACT_TESTCODE.unique()
testcode2id=dict([(testcode_lst[i],i) for i in range(len(testcode_lst))])
#testcode_lst

In [20]:
studyid_textcode_dic=dict()
for studyid,testcode,result in df_labresults[['STUDY_ID','REACT_TESTCODE','PATHOLOGY_RESULT_NUMERIC']].values:
    if studyid not in studyid_textcode_dic:
        studyid_textcode_dic[studyid]=dict()
    if testcode not in studyid_textcode_dic[studyid]:
        studyid_textcode_dic[studyid][testcode]=[]
    studyid_textcode_dic[studyid][testcode].append(result)
#studyid_textcode_dic

In [21]:
labresults_dic=dict()
for studyid in studyid_textcode_dic:
    if studyid not in labresults_dic:
        labresults_dic[studyid]=[None]*len(testcode_lst)*3
    for testcode in studyid_textcode_dic[studyid]:
        results=studyid_textcode_dic[studyid][testcode]
        labresults_dic[studyid][testcode2id[testcode]*3:(testcode2id[testcode]+1)*3]=[np.min(results),np.max(results),np.mean(results)]
#labresults_dic

In [22]:
df_studyid_testcode_result=pd.DataFrame(
    data=[[studyid]+labresults_dic[studyid] for studyid in labresults_dic],
    columns=["STUDY_ID"]+[testcode+suffix for testcode in testcode_lst for suffix in ['_min','_max','_mean']]
)
#df_studyid_testcode_result

In [23]:
#df_studyid_testcode_result.info()

In [24]:
df_patient_features=df_demographics[['STUDY_ID','PATIENT_AGE','GENDER','ETHNIC_GROUP','IS_PREGNANT']].merge(
    df_studyid_testcode_result,how='inner',on='STUDY_ID'
)
#df_patient_features

In [25]:
#df_patient_features.info()

Merge dataframes, fill in default value (0.0) for $\textbf{NaN}$ value in columns ITU_DAYS and AIR_VENT_DEGREE

In [26]:
df_patient_features_labels=df_patient_features.merge(
    df_studyid_ITU,how='left',on='STUDY_ID'
).fillna({'ITU_DAYS':0}).merge(
    df_studyid_air_vent,how='left',on='STUDY_ID'
).fillna({'AIR_VENT_DEGREE':0.0})
#df_patient_features_labels

In [27]:
df_patient_features_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 0 to 323
Data columns (total 67 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   STUDY_ID          324 non-null    object 
 1   PATIENT_AGE       324 non-null    int64  
 2   GENDER            324 non-null    object 
 3   ETHNIC_GROUP      324 non-null    object 
 4   IS_PREGNANT       324 non-null    int64  
 5   GLUCOSE_min       256 non-null    float64
 6   GLUCOSE_max       256 non-null    float64
 7   GLUCOSE_mean      256 non-null    float64
 8   POTASSIUM_min     311 non-null    float64
 9   POTASSIUM_max     311 non-null    float64
 10  POTASSIUM_mean    311 non-null    float64
 11  CREATENINE_min    312 non-null    float64
 12  CREATENINE_max    312 non-null    float64
 13  CREATENINE_mean   312 non-null    float64
 14  LDH_min           194 non-null    float64
 15  LDH_max           194 non-null    float64
 16  LDH_mean          194 non-null    float64
 1

In [28]:
df_patient_features_labels.columns

Index(['STUDY_ID', 'PATIENT_AGE', 'GENDER', 'ETHNIC_GROUP', 'IS_PREGNANT',
       'GLUCOSE_min', 'GLUCOSE_max', 'GLUCOSE_mean', 'POTASSIUM_min',
       'POTASSIUM_max', 'POTASSIUM_mean', 'CREATENINE_min', 'CREATENINE_max',
       'CREATENINE_mean', 'LDH_min', 'LDH_max', 'LDH_mean', 'UREA_min',
       'UREA_max', 'UREA_mean', 'BILIRUBIN_min', 'BILIRUBIN_max',
       'BILIRUBIN_mean', 'WBC_min', 'WBC_max', 'WBC_mean', 'EOSINOPHILS_min',
       'EOSINOPHILS_max', 'EOSINOPHILS_mean', 'NEUTROPHILS_min',
       'NEUTROPHILS_max', 'NEUTROPHILS_mean', 'LYMPHOCYTES_min',
       'LYMPHOCYTES_max', 'LYMPHOCYTES_mean', 'ALT_min', 'ALT_max', 'ALT_mean',
       'CRP_min', 'CRP_max', 'CRP_mean', 'SODIUM_min', 'SODIUM_max',
       'SODIUM_mean', 'HB_min', 'HB_max', 'HB_mean', 'TRIGYCERIN_min',
       'TRIGYCERIN_max', 'TRIGYCERIN_mean', 'PLATELETS_min', 'PLATELETS_max',
       'PLATELETS_mean', 'AST_min', 'AST_max', 'AST_mean', 'TROPONIN_min',
       'TROPONIN_max', 'TROPONIN_mean', 'FERRITIN_min', 'F

X Feature Preprocessing, OneHotEncoding for Categorical features

In [29]:
X_feature_name_lst=[]

In [30]:
from sklearn.preprocessing import OneHotEncoder

X_categorical=df_patient_features_labels[['GENDER', 'ETHNIC_GROUP', 'IS_PREGNANT']]

enc = OneHotEncoder(handle_unknown='ignore')
X_onehot=enc.fit_transform(X_categorical).toarray()

X_feature_name_lst+=list(enc.get_feature_names(['GENDER', 'ETHNIC_GROUP', 'IS_PREGNANT']))
#X_feature_name_lst

In [31]:
numeric_features_lst=[
    'PATIENT_AGE',
    'POTASSIUM_min', 'POTASSIUM_max', 'POTASSIUM_mean', 'ALT_min',
    'ALT_max', 'ALT_mean', 'BILIRUBIN_min', 'BILIRUBIN_max',
    'BILIRUBIN_mean', 'UREA_min', 'UREA_max', 'UREA_mean', 'CREATENINE_min',
    'CREATENINE_max', 'CREATENINE_mean', 'SODIUM_min', 'SODIUM_max',
    'SODIUM_mean', 'CRP_min', 'CRP_max', 'CRP_mean', 'NEUTROPHILS_min',
    'NEUTROPHILS_max', 'NEUTROPHILS_mean', 'EOSINOPHILS_min',
    'EOSINOPHILS_max', 'EOSINOPHILS_mean', 'HB_min', 'HB_max', 'HB_mean',
    'LYMPHOCYTES_min', 'LYMPHOCYTES_max', 'LYMPHOCYTES_mean', 'WBC_min',
    'WBC_max', 'WBC_mean', 'PLATELETS_min', 'PLATELETS_max',
    'PLATELETS_mean', 'GLUCOSE_min', 'GLUCOSE_max', 'GLUCOSE_mean',
    'TRIGYCERIN_min', 'TRIGYCERIN_max', 'TRIGYCERIN_mean', 'FERRITIN_min',
    'FERRITIN_max', 'FERRITIN_mean', 'AST_min', 'AST_max', 'AST_mean',
    'TROPONIN_min', 'TROPONIN_max', 'TROPONIN_mean', 'LDH_min', 'LDH_max',
    'LDH_mean', 'D_DIMER_min', 'D_DIMER_max', 'D_DIMER_mean'
]

X_feature_name_lst+=numeric_features_lst

#X_feature_name_lst

In [32]:
X_numeric=df_patient_features_labels[numeric_features_lst].values
#X_numeric

In [33]:
X_missing=np.concatenate([X_onehot, X_numeric], axis=1)
#X_missing

fill in missing features value with mean value

In [34]:
#from sklearn.impute import KNNImputer

#imputer = KNNImputer(n_neighbors=5, weights="uniform")

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

X=imputer.fit_transform(X_missing)

#X

Two Y labels ITU_DAYS, AIR_VENTILATOR

In [35]:
Y_ITU=df_patient_features_labels.values[:,-2]
Y_AIR=df_patient_features_labels.values[:,-1]

ITU_DAYS prediction test, evaluated by R2_score:
$R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

models={
    "Linear":LinearRegression(),
    "MLP":MLPRegressor(max_iter=3000,solver='adam', alpha=1e-3,hidden_layer_sizes=(64, 16)),
    "DecisionTree":DecisionTreeRegressor(),
    "RandomForest":RandomForestRegressor(),
    "AdaBoost":AdaBoostRegressor(n_estimators=100),
    "GradientBoosting":GradientBoostingRegressor(),
}

In [37]:
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score

r2_result_lst=[]

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y_ITU[train_index], Y_ITU[test_index]
    r2_result_lst.append(
        [r2_score(y_test,models[model_name].fit(X_train,y_train).predict(X_test))
         for model_name in models])

print(pd.DataFrame(r2_result_lst,columns=[model_name for model_name in models]))

     Linear        MLP  DecisionTree  RandomForest  AdaBoost  GradientBoosting
0 -0.792547  -4.714742     -0.297665      0.627936  0.710132          0.688121
1 -0.335223  -1.626195     -0.320574      0.221589  0.376945          0.282592
2 -0.502486  -8.570367      0.004233      0.651726  0.605825          0.726052
3 -0.510717 -23.386230      0.579502      0.559848  0.678150          0.542300
4  0.425073   0.007565      0.610680      0.579813  0.680813          0.621091


ITU_DAYS feature importance analysis $\textbf{RandomForest}$

In [38]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 6.433440344849707e-05
GENDER_M : 7.261477383083096e-05
ETHNIC_GROUP_A : 0.00037876501945662385
ETHNIC_GROUP_C : 5.7778231693406466e-05
ETHNIC_GROUP_E : 0.0
ETHNIC_GROUP_G : 0.00045939329812811285
ETHNIC_GROUP_H : 4.844977022282261e-06
ETHNIC_GROUP_J : 0.0
ETHNIC_GROUP_L : 0.00010811632218219952
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 0.004248366400712536
ETHNIC_GROUP_P : 0.0
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 1.3694437137135315e-06
ETHNIC_GROUP_X : 1.4012014177736512e-07
ETHNIC_GROUP_Z : 2.6445024988890913e-06
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.0019068789713521234
POTASSIUM_min : 0.0012259032030114603
POTASSIUM_max : 0.007682786540044607
POTASSIUM_mean : 0.0014295487502811028
ALT_min : 0.0025530713844611076
ALT_max : 0.005309527431879575
ALT_mean : 0.003970225569225537
BILIRUBIN_min : 0.004353918832012843
BILIRUBIN_max : 0.004155570594662947
BILIRUBIN_mean : 0.003073412959583481
UREA_min : 0.0014950094823848387
UREA_max : 0.008048221222692333
UREA_mean : 0.0066182234824375

In [39]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('TRIGYCERIN_max', 0.17631621121199725),
 ('EOSINOPHILS_max', 0.16242332331673287),
 ('CRP_max', 0.1606136108336944),
 ('AST_min', 0.10905284877296023),
 ('CREATENINE_min', 0.10468665594920573),
 ('TRIGYCERIN_min', 0.045494123753538486),
 ('D_DIMER_max', 0.02197373203853222),
 ('HB_min', 0.015370162499182778)]

ITU_DAYS feature importance analysis $\textbf{AdaBoost}$

In [40]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 0.0010478080277309782
GENDER_M : 0.002632501000218817
ETHNIC_GROUP_A : 0.0004925628849268629
ETHNIC_GROUP_C : 0.0
ETHNIC_GROUP_E : 0.0
ETHNIC_GROUP_G : 0.0006369132584149704
ETHNIC_GROUP_H : 0.0
ETHNIC_GROUP_J : 0.0
ETHNIC_GROUP_L : 3.221984916653279e-05
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 0.0011012224337766641
ETHNIC_GROUP_P : 0.0
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 0.0
ETHNIC_GROUP_X : 0.0
ETHNIC_GROUP_Z : 0.0
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.002098245668071076
POTASSIUM_min : 0.003983586258045416
POTASSIUM_max : 0.027559948687774973
POTASSIUM_mean : 0.009479288684817533
ALT_min : 2.357666635790126e-05
ALT_max : 0.0036731811413866534
ALT_mean : 0.0001436958670585267
BILIRUBIN_min : 0.004219412942029145
BILIRUBIN_max : 0.00039249214629741123
BILIRUBIN_mean : 0.0052465849867663625
UREA_min : 0.004328110840961709
UREA_max : 0.0044091492486334795
UREA_mean : 0.0025868218823197012
CREATENINE_min : 0.08724394361760562
CREATENINE_max : 0.001927820310289605
CREATENINE_mean 

In [41]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('AST_min', 0.19418656587042812),
 ('CRP_max', 0.11663444477906902),
 ('D_DIMER_max', 0.1016478349468603),
 ('CREATENINE_min', 0.08724394361760562),
 ('HB_min', 0.08435466207340028),
 ('TRIGYCERIN_min', 0.04866607822339483),
 ('EOSINOPHILS_mean', 0.037088195721190084),
 ('TRIGYCERIN_max', 0.03512245755852078),
 ('POTASSIUM_max', 0.027559948687774973),
 ('FERRITIN_min', 0.026435871226627987)]

ITU_DAYS feature importance analysis $\textbf{GradientBoostingTree}$

In [42]:
from sklearn.ensemble import GradientBoostingRegressor

regr = GradientBoostingRegressor().fit(X, Y_ITU)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 0.0
GENDER_M : 0.0
ETHNIC_GROUP_A : 1.324382922337909e-07
ETHNIC_GROUP_C : 0.0
ETHNIC_GROUP_E : 0.0
ETHNIC_GROUP_G : 0.0
ETHNIC_GROUP_H : 0.0
ETHNIC_GROUP_J : 0.0
ETHNIC_GROUP_L : 0.0
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 0.00108606544855589
ETHNIC_GROUP_P : 0.0
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 0.0
ETHNIC_GROUP_X : 0.0
ETHNIC_GROUP_Z : 0.0
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.00178257292094354
POTASSIUM_min : 0.00018886192950387526
POTASSIUM_max : 0.0077191260574679685
POTASSIUM_mean : 0.006417192529975659
ALT_min : 3.056781406180897e-05
ALT_max : 0.00022361571068076945
ALT_mean : 0.0002964770634966257
BILIRUBIN_min : 0.011004364140185446
BILIRUBIN_max : 8.79281485984589e-05
BILIRUBIN_mean : 0.0008901996562796411
UREA_min : 2.96942231165821e-05
UREA_max : 0.0008135932562444776
UREA_mean : 0.010635547687908635
CREATENINE_min : 0.06742086712509641
CREATENINE_max : 0.0010507935911863925
CREATENINE_mean : 0.0008253105306256921
SODIUM_min : 0.00032330779492044555
SODIUM_max :

In [43]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('EOSINOPHILS_max', 0.2866979821311893),
 ('CRP_max', 0.16919646636806723),
 ('TRIGYCERIN_max', 0.13347126458905825),
 ('D_DIMER_max', 0.08838142630323106),
 ('CREATENINE_min', 0.06742086712509641),
 ('TRIGYCERIN_min', 0.06445438184066717),
 ('AST_min', 0.021383517457617834),
 ('FERRITIN_mean', 0.017084797137337324),
 ('EOSINOPHILS_mean', 0.01363177907562766)]

AIR_ventilator prediction test, evaluated by R2_score:
$R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

models={
    "Linear":LinearRegression(),
    "MLP":MLPRegressor(max_iter=3000,solver='adam', alpha=1e-3,hidden_layer_sizes=(64, 16)),
    "DecisionTree":DecisionTreeRegressor(),
    "RandomForest":RandomForestRegressor(),
    "AdaBoost":AdaBoostRegressor(n_estimators=100),
    "GradientBoosting":GradientBoostingRegressor(),
}

In [45]:
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score

r2_result_lst=[]

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y_AIR[train_index], Y_AIR[test_index]
    r2_result_lst.append(
        [r2_score(y_test,models[model_name].fit(X_train,y_train).predict(X_test))
         for model_name in models])

print(pd.DataFrame(r2_result_lst,columns=[model_name for model_name in models]))

     Linear        MLP  DecisionTree  RandomForest  AdaBoost  GradientBoosting
0  0.028640  -9.329925     -0.158850      0.490783  0.591655          0.539969
1  0.167084 -28.127425     -0.043197      0.625591  0.516396          0.595089
2 -0.110639 -85.170361      0.385280      0.560576  0.535526          0.518085
3 -1.839184 -25.543466     -0.007564      0.550191  0.489163          0.580513
4  0.249323 -69.201009      0.382014      0.554062  0.487554          0.512206


AIR_ventilator feature importance analysis $\textbf{RandomForest}$

In [46]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 0.00131806967929291
GENDER_M : 0.0006874752667959194
ETHNIC_GROUP_A : 0.00017081372416345952
ETHNIC_GROUP_C : 3.5605945757458525e-06
ETHNIC_GROUP_E : 1.4551632732521222e-06
ETHNIC_GROUP_G : 0.0
ETHNIC_GROUP_H : 0.0005454683866574066
ETHNIC_GROUP_J : 9.648867879511102e-06
ETHNIC_GROUP_L : 0.0007449493534004935
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 3.0539401318819736e-06
ETHNIC_GROUP_P : 0.0
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 5.724335291117744e-05
ETHNIC_GROUP_X : 1.883409186055167e-05
ETHNIC_GROUP_Z : 0.00011306511411892086
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.05536129935370548
POTASSIUM_min : 0.005273438086296391
POTASSIUM_max : 0.024725498678445278
POTASSIUM_mean : 0.0119093568982215
ALT_min : 0.003231333846442641
ALT_max : 0.026611242788302705
ALT_mean : 0.009452312304526584
BILIRUBIN_min : 0.009261182263906313
BILIRUBIN_max : 0.005774223424742344
BILIRUBIN_mean : 0.005585904108086863
UREA_min : 0.005961414221523069
UREA_max : 0.008314450976499266
UREA_mean : 0.008061948

In [47]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('TRIGYCERIN_max', 0.24425127783893916),
 ('CRP_max', 0.13341916593213507),
 ('LDH_max', 0.07808134336885582),
 ('D_DIMER_max', 0.06679783238990399),
 ('PATIENT_AGE', 0.05536129935370548),
 ('ALT_max', 0.026611242788302705),
 ('POTASSIUM_max', 0.024725498678445278),
 ('CRP_mean', 0.021664207843494775),
 ('D_DIMER_min', 0.02074052939140987),
 ('TRIGYCERIN_min', 0.01779003778138871)]

AIR_ventilator feature importance analysis $\textbf{AdaBoost}$

In [48]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 0.0
GENDER_M : 0.0
ETHNIC_GROUP_A : 0.0
ETHNIC_GROUP_C : 0.0
ETHNIC_GROUP_E : 0.0
ETHNIC_GROUP_G : 0.0
ETHNIC_GROUP_H : 0.0
ETHNIC_GROUP_J : 0.0
ETHNIC_GROUP_L : 0.001440147672738927
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 0.0
ETHNIC_GROUP_P : 0.001084266355854827
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 0.0
ETHNIC_GROUP_X : 0.0014029315270668494
ETHNIC_GROUP_Z : 0.0
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.09013949404082458
POTASSIUM_min : 0.004600125263586607
POTASSIUM_max : 0.04208707452882776
POTASSIUM_mean : 0.020747723968477374
ALT_min : 0.001587695886093182
ALT_max : 0.029327354506280157
ALT_mean : 0.015343530334602961
BILIRUBIN_min : 0.015702447243113114
BILIRUBIN_max : 0.003885836061127963
BILIRUBIN_mean : 0.00928358890085354
UREA_min : 0.0
UREA_max : 0.008550146806006478
UREA_mean : 0.015527785682248605
CREATENINE_min : 0.005238485763429345
CREATENINE_max : 0.015529674858001783
CREATENINE_mean : 0.0012578491408903245
SODIUM_min : 0.007709314522535455
SODIUM_max : 0.019035609

In [49]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('PATIENT_AGE', 0.09013949404082458),
 ('CRP_max', 0.07674396608847357),
 ('D_DIMER_max', 0.0682261729384314),
 ('CRP_mean', 0.0519331336500658),
 ('GLUCOSE_min', 0.04309408696091272),
 ('POTASSIUM_max', 0.04208707452882776),
 ('TROPONIN_min', 0.040873284177569996),
 ('D_DIMER_min', 0.03690454424647643),
 ('SODIUM_mean', 0.03635027942494197),
 ('GLUCOSE_mean', 0.03447844410643994)]

AIR_ventilator feature importance analysis $\textbf{GradientBoostingTree}$

In [50]:
from sklearn.ensemble import GradientBoostingRegressor

regr = GradientBoostingRegressor().fit(X, Y_AIR)

for feature,importance in zip(X_feature_name_lst,regr.feature_importances_):
    print(feature,":",importance)

GENDER_F : 0.0
GENDER_M : 0.0008300650881879382
ETHNIC_GROUP_A : 0.00046738648657812374
ETHNIC_GROUP_C : 0.0
ETHNIC_GROUP_E : 0.0
ETHNIC_GROUP_G : 0.0
ETHNIC_GROUP_H : 0.0
ETHNIC_GROUP_J : 0.0
ETHNIC_GROUP_L : 0.0
ETHNIC_GROUP_M : 0.0
ETHNIC_GROUP_N : 0.0
ETHNIC_GROUP_P : 0.00038113485681811634
ETHNIC_GROUP_R : 0.0
ETHNIC_GROUP_S : 0.0
ETHNIC_GROUP_X : 0.0
ETHNIC_GROUP_Z : 0.0
IS_PREGNANT_0 : 0.0
PATIENT_AGE : 0.061539809601234396
POTASSIUM_min : 0.00025582446263704606
POTASSIUM_max : 0.04449617401998308
POTASSIUM_mean : 0.002363344632184042
ALT_min : 0.0015149455352347905
ALT_max : 0.019964237384961284
ALT_mean : 0.007869757228297646
BILIRUBIN_min : 0.0014613348779521597
BILIRUBIN_max : 0.0022428149952115235
BILIRUBIN_mean : 0.0003183665173366538
UREA_min : 0.002378855197518968
UREA_max : 0.009417837889875089
UREA_mean : 0.007193129011382494
CREATENINE_min : 0.006673816803359852
CREATENINE_max : 0.0007375637969271162
CREATENINE_mean : 0.015967237504950528
SODIUM_min : 0.00830094396062

In [51]:
feature_importance_dic=dict(zip(X_feature_name_lst,regr.feature_importances_))
from sklearn.feature_selection import SelectFromModel
sorted(
    [(X_feature_name_lst[i],feature_importance_dic[X_feature_name_lst[i]]) for i in SelectFromModel(regr, prefit=True, max_features=10).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
)

[('TRIGYCERIN_max', 0.24045953932748726),
 ('CRP_max', 0.1393858395720832),
 ('D_DIMER_max', 0.06486643792560656),
 ('PATIENT_AGE', 0.061539809601234396),
 ('TROPONIN_min', 0.056236351205846136),
 ('POTASSIUM_max', 0.04449617401998308),
 ('LDH_max', 0.04171311413382089),
 ('SODIUM_max', 0.03147763230287275),
 ('CRP_min', 0.023996623971727055),
 ('GLUCOSE_min', 0.023251774799755074)]